## This guide gets you started programming in the low-level TensorFlow APIs (TensorFlow Core), showing you how to:

* Manage your own TensorFlow program (a tf.Graph) and TensorFlow runtime (a tf.Session), instead of relying on Estimators to manage them.
* Run TensorFlow operations, using a tf.Session.
* Use high level components (datasets, layers, and feature_columns) in this low level environment.
* Build your own training loop, instead of using the one provided by Estimators.

## We recommend using the higher level APIs to build models when possible. Knowing TensorFlow Core is valuable for the following reasons:

* Experimentation and debugging are both more straight forward when you can use low level TensorFlow operations directly.
* It gives you a mental model of how things work internally when using the higher level APIs.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
print(tf.__version__)

1.14.0


# Tensors

TensorFlow, as the name indicates, is a framework to define and run computations involving tensors. A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes. TensorFlow programs work by first building a graph of tf.Tensor objects, detailing how each tensor is computed based on the other available tensors and then by running parts of this graph to achieve the desired results. A tf.Tensor has the following properties:
* a data type (float32, int32, or string, for example)
* a shape

Each element in the Tensor has the same data type, and the data type is always known. Some types of tensors are special, and these will be covered in other units of the TensorFlow guide. The main ones are:
* tf.Variable
* tf.constant
* tf.placeholder
* tf.SparseTensor

With the exception of tf.Variable, the value of a tensor is immutable, which means that in the context of a single execution tensors only have a single value.

**The rank of a tf.Tensor object is its number of dimensions.**

* Rank 0 --> Scalar (magnitude only)


In [2]:
mammal = tf.Variable("Elephant", tf.string) # A string is treated as a single object in TensorFlow, not as a sequence of characters. 
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable(12.3 - 4.85j, tf.complex64)

* 1 --> Vector (magnitude and direction)

In [3]:
mystr = tf.Variable(["Hello"], tf.string)
cool_numbers  = tf.Variable([3.14159, 2.71828], tf.float32)
first_primes = tf.Variable([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.Variable([12.3 - 4.85j, 7.5 - 6.23j], tf.complex64)

* 2 --> Matrix (table of numbers)

In [4]:
mymat = tf.Variable([[7],[11]], tf.int16)
myxor = tf.Variable([[False, True],[True, False]], tf.bool)
linear_squares = tf.Variable([[4], [9], [16], [25]], tf.int32)
squarish_squares = tf.Variable([ [4, 9], [16, 25] ], tf.int32)

* 3 -->3-Tensor (cube of numbers)
* n --> n-Tensor an n-dimensional array

In [5]:
my_image = tf.zeros([10, 224, 224, 3])  # batch x height x width x channels

**tf.Tensor object's rank**

In [6]:
r = tf.rank(my_image)
# After the graph runs, r will hold the value 4.

**tf.tensor slices**

In [7]:
first_image = my_image[1,:,:,:]

**tf.tensor object's shape and reshape**

In [8]:
zeros = tf.zeros(first_image.shape[1])

rank_three_tensor = tf.ones([3, 4, 5])
# Reshape existing content into a 6x10 matrix
matrix = tf.reshape(rank_three_tensor, [6, 10])  
#  Reshape existing content into a 3x20 matrix.
matrixB = tf.reshape(matrix, [3, -1])  
# Reshape existing content into a 4x3x5 tensor
matrixAlt = tf.reshape(matrixB, [4, 3, -1]) 

**tf.tensor object's data types**

It is not possible to have a tf.Tensor with more than one data type. It is possible, however, to serialize arbitrary data structures as strings and store those in tf.Tensors. It is possible to cast tf.Tensors from one datatype to another using tf.cast.

In [9]:
float_tensor = tf.cast(tf.constant([1, 2, 3]), dtype=tf.float32)

**Creating a variable and assigning it to a collection**

TensorFlow provides collections, which are named lists of tensors or other objects, such as tf.Variable instances.

By default every tf.Variable gets placed in the following two collections:

* tf.GraphKeys.GLOBAL_VARIABLES --- variables that can be shared across multiple devices,
* tf.GraphKeys.TRAINABLE_VARIABLES --- variables for which TensorFlow will calculate gradients.
* tf.GraphKeys.LOCAL_VARIABLES  --- If you don't want a variable to be trainable

`my_local = tf.get_variable("my_local", shape=(),collections=[tf.GraphKeys.LOCAL_VARIABLES])`

`# --or--`

`my_non_trainable = tf.get_variable("my_non_trainable",shape=(),trainable=False)`

`# by default the dtype is tf.float32 and its initial value will be randomized via tf.glorot_uniform_initializer.`

**Device placement of variable**

`with tf.device("/device:GPU:1"): v = tf.get_variable("v", [1])`

**Initializing variables**

Before you can use a variable, it must be initialized. If you are programming in the low-level TensorFlow API (that is, you are explicitly creating your own graphs and sessions), you must explicitly initialize the variables. Most high-level frameworks such as tf.contrib.slim, tf.estimator.Estimator and Keras automatically initialize variables for you before training a model.

`session.run(tf.global_variables_initializer())`

`# Now all variables in the tf.GraphKeys.GLOBAL_VARIABLES collection are initialized`

**Sharing variables**

TensorFlow supports two ways of sharing variables:

* Explicitly passing tf.Variable objects around.
* Implicitly wrapping tf.Variable objects within tf.variable_scope objects.

While code which explicitly passes variables around is very clear, it is sometimes convenient to write TensorFlow functions that implicitly use variables in their implementations. Most of the functional layers from tf.layers use this approach, as well as all tf.metrics, and a few other library utilities.



In [10]:
def conv_relu(input, kernel_shape, bias_shape):
    
    #Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
    
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    
    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

# This function uses short names weights and biases, which is good for clarity. In a real model, 
# however, we want many such convolutional layers, and calling this function repeatedly would not work.

input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])

with tf.variable_scope("model") as scope:
    # Variables created here will be named "model/weights", "model/biases".
    x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
with tf.variable_scope(scope, reuse=True):
    output = conv_relu(x, [5, 5, 32, 32], [32])

# Graph and Sessions

TensorFlow uses a dataflow graph to represent your computation in terms of the dependencies between individual operations. This leads to a low-level programming model in which you first define the dataflow graph, then create a TensorFlow session to run parts of the graph across a set of local and remote devices. Higher-level APIs such as tf.estimator.Estimator and Keras hide the details of graphs and sessions from the end user, but this guide may also be useful if you want to understand how these APIs are implemented.

Dataflow is a common programming model for parallel computing. In a dataflow graph, the nodes represent units of computation, and the edges represent the data consumed or produced by a computation. For example, in a TensorFlow graph, the tf.matmul operation would correspond to a single node with two incoming edges (the matrices to be multiplied) and one outgoing edge (the result of the multiplication).

Dataflow has several advantages that TensorFlow leverages when executing your programs:

* Parallelism. By using explicit edges to represent dependencies between operations, it is easy for the system to identify operations that can execute in parallel.

* Distributed execution. By using explicit edges to represent the values that flow between operations, it is possible for TensorFlow to partition your program across multiple devices (CPUs, GPUs, and TPUs) attached to different machines. TensorFlow inserts the necessary communication and coordination between devices.

* Compilation. TensorFlow's XLA compiler can use the information in your dataflow graph to generate faster code, for example, by fusing together adjacent operations.

* Portability. The dataflow graph is a language-independent representation of the code in your model. You can build a dataflow graph in Python, store it in a SavedModel, and restore it in a C++ program for low-latency inference.

A tf.Graph contains two relevant kinds of information:

* Graph structure. The nodes and edges of the graph, indicating how individual operations are composed together, but not prescribing how they should be used.
* Graph collections. TensorFlow provides a general mechanism for storing collections of metadata in a tf.Graph. 

Most TensorFlow programs start with a dataflow graph construction phase. In this phase, you invoke TensorFlow API functions that construct new tf.Operation (node) and tf.Tensor (edge) objects and add them to a tf.Graph instance. TensorFlow provides a default graph that is an implicit argument to all API functions in the same context. For ex:

* Calling tf.matmul(x, y) creates a single tf.Operation that multiplies the values of tf.Tensor objects x and y, adds it to the default graph, and returns a tf.Tensor that represents the result of the multiplication.

A tf.Graph object defines a namespace for the tf.Operation objects it contains. TensorFlow automatically chooses a unique name for each operation in your graph, but giving operations descriptive names can make your program easier to read and debug. 

If you want your TensorFlow program to use multiple different devices, the tf.device function provides a convenient way to request that all operations created in a particular context are placed on the same device (or type of device).

A device specification has the following form: /job:<JOB_NAME>/task:<TASK_INDEX>/device:<DEVICE_TYPE>:<DEVICE_INDEX>

tf.device gives you a lot of flexibility to choose placements for individual operations or broad regions of a TensorFlow graph. In many cases, there are simple heuristics that work well. For example, the tf.train.replica_device_setter API can be used with tf.device to place operations for data-parallel distributed training. 

`with tf.device(tf.train.replica_device_setter(ps_tasks=3)):`
  
  `# tf.Variable objects are, by default, placed on tasks in "/job:ps" in a round-robin fashion.`
    
    w_0 = tf.Variable(...)  # placed on "/job:ps/task:0"
    b_0 = tf.Variable(...)  # placed on "/job:ps/task:1"
    w_1 = tf.Variable(...)  # placed on "/job:ps/task:2"
    b_1 = tf.Variable(...)  # placed on "/job:ps/task:0"

    input_data = tf.placeholder(tf.float32)     # placed on "/job:worker"
    layer_0 = tf.matmul(input_data, w_0) + b_0  # placed on "/job:worker"
    layer_1 = tf.matmul(layer_0, w_1) + b_1     # placed on "/job:worker"

Tensor-like objects include elements of the following types: tf.Tensor, tf.Variable, numpy.ndarray, list (and lists of tensor-like objects), Scalar Python types: bool, float, int, str

A tf.Session object provides access to devices in the local machine, and remote devices using the distributed TensorFlow runtime. It also caches information about your tf.Graph so that you can efficiently run the same computation multiple times. Since a tf.Session owns physical resources (such as GPUs and network connections), it is typically used as a context manager (in a with block) that automatically closes the session when you exit the block.

tf.Session.init accepts three optional arguments:
* target - If this argument is left empty (the default), the session will only use devices in the local machine. 
* graph - By default, a new tf.Session will be bound to---and only able to run operations in---the current default graph. 
* config - This argument allows you to specify a tf.ConfigProto that controls the behavior of the session, ex: graph_options.optimizer_options, gpu_options.allow_growth

The tf.Session.run method is the main mechanism for running a tf.Operation or evaluating a tf.Tensor. You can pass one or more tf.Operation or tf.Tensor objects to tf.Session.run, and TensorFlow will execute the operations that are needed to compute the result.

In [11]:
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
output = tf.nn.softmax(y)

init_op = w.initializer
with tf.Session() as sess:
  
    # Run the initializer on `w`.
    sess.run(init_op)

    # Evaluate `output`. `sess.run(output)` will return a NumPy array containing
    # the result of the computation.
    print(sess.run(output))

    # Evaluate `y` and `output`. Note that `y` will only be computed once, and its
    # result used both to return `y_val` and as an input to the `tf.nn.softmax()`
    # op. Both `y_val` and `output_val` will be NumPy arrays.
    y_val, output_val = sess.run([y, output])

[[9.9994397e-01 5.6010675e-05]
 [5.7035828e-01 4.2964175e-01]]


You may not always have the constants right away, and you may be waiting for a constant to appear after a cycle of operations. **tf.placeholder** is a tool for this. It inserts a placeholder for a tensor that will be always fed.

In [12]:
x= tf.placeholder(tf.float32)
y= tf.placeholder(tf.float32)

add= tf.add(x,y)
sub= tf.subtract(x,y)
mult=tf.multiply(x,y)
div= tf.divide(x,y)

d={x:20.,y:10.}

In [13]:
with tf.Session() as sess:
    # `sess.graph` provides access to the graph used in a `tf.Session`.
    #writer = tf.summary.FileWriter(".", sess.graph)
    print('Addition',sess.run(add,feed_dict=d))
    print('Subtraction',sess.run(sub,feed_dict=d))
    print('Multiplication',sess.run(mult,feed_dict=d))
    print('Division',sess.run(div,feed_dict=d))

Addition 30.0
Subtraction 10.0
Multiplication 200.0
Division 2.0
